In [ ]:
import speech_recognition as sr
from gtts import gTTS
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import keyboard

mood_data_x = []
mood_data_y = []
counter = 0
all_words = ""

tfidf = np.load('TF-IDF.npy', allow_pickle=True).item()
log_reg = np.load( 'IMDB_log_reg.npy', allow_pickle=True).item()

# Initialize the recognizer
r = sr.Recognizer()

# Function to convert text to speech using gTTS
def SpeakText(command):
    tts = gTTS(text=command, lang='en')
    tts.save("output.mp3")
    os.system("afplay output.mp3")  # For macOS, plays the saved audio file

# Loop infinitely for the user to speak
while True:
    # Exception handling to handle exceptions at runtime
    try:
        if keyboard.read_key() == "q":  # if key 'q' is pressed 
            break
        
        # Use the microphone as the source for input.
        with sr.Microphone() as source:
            # Wait for a second to let the recognizer adjust the energy threshold based on the surrounding noise level
            r.adjust_for_ambient_noise(source, duration=0.2)
            
            # Listen for the user's input
            audio = r.listen(source)
            
            # Using Google to recognize audio
            MyText = r.recognize_google(audio)
            MyText = MyText.lower()
            
            if "stop recording" in MyText:
                print("session stopped")
                break
                
            print("Did you say:", MyText)
            
            input_vector = tfidf.transform([MyText])
            
            # Get predicted probabilities for each class
            predicted_probabilities = log_reg.predict_proba(input_vector)
            
            # Make final predictions
            final_predictions = log_reg.predict(input_vector)
            print("Final Predictions:", final_predictions)
            
            mood_data_x.append(counter)
            counter += 1
            mood_data_y.append(predicted_probabilities[0][1])
            all_words += MyText + " "
            
    except sr.RequestError as e:
        print("Could not request results:", e)
        
    except sr.UnknownValueError:
        print("say something")
        continue
        
# plot probability data from recording
plt.plot(mood_data_x,mood_data_y)
plt.xlabel("Time")
plt.ylabel("Happiness")
plt.show()
final_predictions = log_reg.predict(input_vector)
print("Overall your story was... ", final_predictions)

Exception in thread Thread-5 (listen):
Traceback (most recent call last):
  File "//anaconda3/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "//anaconda3/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "//anaconda3/lib/python3.10/site-packages/keyboard/__init__.py", line 294, in listen
    _os_keyboard.listen(self.direct_callback)
  File "//anaconda3/lib/python3.10/site-packages/keyboard/_darwinkeyboard.py", line 430, in listen
    raise OSError("Error 13 - Must be run as administrator")
OSError: Error 13 - Must be run as administrator
